In [1]:
# from pymagnitude import Magnitude

In [2]:
# glove = Magnitude('data/magnitude/glove.6B.300d.magnitude')

In [3]:
# fasttext = Magnitude('data/magnitude/wiki-news-300d-1M-subword.magnitude', case_insensitive=True)

In [4]:
# print(glove.dim)
# print(fasttext.dim)

In [5]:
# print(glove[0])

In [6]:
# glove.query(["I", "read", "a", "book"])

In [7]:
# fasttext.query([["I", "read", "a", "book"], ["I", "read", "a", "magazine"]])

In [8]:
# vectors = Magnitude(glove, fasttext)

In [9]:
# vectors.query(["I", "read", "a", "book"])

In [10]:
# vectors.query([["I", "read", "a", "book"], ["I", "read", "a", "magazine"]])

In [11]:
# print(vectors.dim)
# emdim = vectors.dim

In [12]:
# from keras import backend as K

In [13]:
# K.print()

In [14]:
# import tensorflow as tf
# from keras import backend as K
# from tensorflow.python import debug as tf_debug
# K.set_session(tf_debug.LocalCLIDebugWrapperSession(tf.Session()))

In [1]:
emdim = 600

In [2]:
from keras.layers import Input, TimeDistributed, LSTM, Bidirectional, Lambda
from keras.models import Model
from layers import Highway, Similarity, C2QAttention, Q2CAttention, MergedContext, SpanBegin, SpanEnd, CombineOutputs
from scripts import negative_avg_log_error, BatchGenerator

Using TensorFlow backend.


In [3]:
question_input = Input(shape=(None, emdim), dtype='float32', name="question_input")
passage_input = Input(shape=(None, emdim), dtype='float32', name="passage_input")

In [4]:
# def highway_layer(value, activation="tanh", transform_gate_bias=-1.0):
#     dim = K.int_shape(value)[-1]
#     transform_gate_bias_initializer = keras.initializers.Constant(transform_gate_bias)
#     transform_gate = Dense(units=dim, bias_initializer=transform_gate_bias_initializer)(value)
#     transform_gate = Activation("sigmoid")(transform_gate)
#     carry_gate = Lambda(lambda x: 1.0 - x, output_shape=(dim,))(transform_gate)
#     transformed_data = Dense(units=dim)(value)
#     transformed_data = Activation(activation)(transformed_data)
#     transformed_gated = Multiply()([transform_gate, transformed_data])
#     identity_gated = Multiply()([carry_gate, value])
#     value = Add()([transformed_gated, identity_gated])
#     return value

In [5]:
num_highway_layers = 2

In [6]:
for i in range(num_highway_layers):
    highway_layer = Highway(name='highway_{}'.format(i))
    question_layer = TimeDistributed(highway_layer, name=highway_layer.name + "_qtd")
    question_embedding = question_layer(question_input)
    passage_layer = TimeDistributed(highway_layer, name=highway_layer.name + "_ptd")
    passage_embedding = passage_layer(passage_input)

In [7]:
#from keras import backend as K
#def func(x):
#    return K.eval(K.shape(x))(x, 'question_embedding')
#question_embedding = Lambda(func)(question_embedding)

In [8]:
# a = {'a': 1}
# b = a.get('b')
# print(b)
# b in a

In [9]:
# a = {'a': 1}
# b = a.pop('a')
# print(b)
# print(a)

In [10]:
encoder_layer = Bidirectional(LSTM(emdim, return_sequences=True), name='bidirectional_encoder')
encoded_question = encoder_layer(question_embedding)
encoded_passage = encoder_layer(passage_embedding)

In [11]:
similarity_matrix = Similarity(name='similarity_layer')([encoded_passage, encoded_question])

In [12]:
# import numpy as np
# a = np.random.random((600, 1))
# b = np.random.random((600, 1))
# c = np.dot(a.T, b)
# print(c)

In [13]:
context_to_query_attention = C2QAttention(name='context_to_query_attention')([similarity_matrix, encoded_question])
query_to_context_attention = Q2CAttention(name='query_to_context_attention')([similarity_matrix, encoded_passage])

(None, None)
(None, None, 1200)


In [14]:
merged_context = MergedContext(name='merged_context')([encoded_passage, context_to_query_attention, query_to_context_attention])

In [15]:
num_decoders = 1

In [16]:
modeled_passage = merged_context
for i in range(num_decoders):
    hidden_layer = Bidirectional(LSTM(emdim, return_sequences=True), name='bidirectional_decoder_{}'.format(i))
    modeled_passage = hidden_layer(modeled_passage)

In [17]:
span_begin_probabilities = SpanBegin(name='span_begin')([merged_context, modeled_passage])
span_end_probabilities = SpanEnd(name='span_end')([encoded_passage, merged_context, modeled_passage, span_begin_probabilities])

In [18]:
output = CombineOutputs(name='combine_outputs')([span_begin_probabilities, span_end_probabilities])

In [19]:
model = Model([passage_input, question_input], [output])

In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
passage_input (InputLayer)      (None, None, 600)    0                                            
__________________________________________________________________________________________________
question_input (InputLayer)     (None, None, 600)    0                                            
__________________________________________________________________________________________________
highway_1_ptd (TimeDistributed) (None, None, 600)    0           passage_input[0][0]              
__________________________________________________________________________________________________
highway_1_qtd (TimeDistributed) (None, None, 600)    0           question_input[0][0]             
__________________________________________________________________________________________________
bidirectio

In [21]:
model.compile(loss=negative_avg_log_error, optimizer='adadelta', metrics=[])

In [22]:
train_generator = BatchGenerator('train')
validate_generator = BatchGenerator('dev')

In [23]:
history = model.fit_generator(train_generator, steps_per_epoch=2, epochs=1, validation_data=validate_generator, validation_steps=1, shuffle=False)

Epoch 1/1
2/2 [==============================] - 77s 39s/step - loss: 11.4518 - val_loss: 11.4704


In [24]:
model.save('bidaf')

In [25]:
h = history.history

In [26]:
x,y = train_generator.__getitem__(0)
z = model.predict(x)

In [27]:
import numpy as np
a = z[:,0,:]
b = z[:,1,:]
s = np.sum(a,axis=-1)
v = np.sum(b,axis=-1)
print(s)
print(v)

[1.         0.9999999  1.         1.         0.99999994 1.0000001
 1.0000002  1.         0.9999999  1.         1.         1.
 1.         0.99999964 1.0000001  0.99999976 1.0000002  1.0000002
 1.0000002  1.0000002  1.0000004  1.0000002  1.0000005  1.0000002
 0.99999994 0.99999976 1.0000001  0.9999999  1.0000002  1.
 0.9999998  1.        ]
[1.         1.0000004  0.99999964 0.99999976 1.0000002  0.9999999
 0.9999999  0.99999994 0.9999999  0.99999976 1.0000001  1.
 0.99999994 1.0000005  0.99999976 0.99999964 1.         1.0000004
 1.         1.         0.9999997  1.         1.0000002  0.9999997
 1.         0.99999994 0.9999996  0.9999997  0.9999999  1.0000002
 0.99999964 0.9999996 ]


In [ ]:
# import numpy as np
# a = np.random.random((10, 1))
# b = np.random.random((10, 1))
# # print(a)
# print(b[0])
# for v in a:
#     for w in b:
#         print(w)
# #         c = a*b
# #         print(c)
#         break
#     break

In [ ]:
# print(encoded_question.shape[0])

In [ ]:
# import numpy as np
# from keras.layers import concatenate, multiply, dot, Input


# def concatenate_and_multiply(context_vector, query_vector):
#     element_wise_mul = multiply([context_vector, query_vector], axis=-1)
#     concatenate_vecs = concatenate([context_vector, query_vector, element_wise_mul], axis=-1)
#     return concatenate_vecs


# a = np.random.random((10, 10, 20))
# b = np.random.random((10, 15, 20))
# v = Input(shape=(10, 20))
# w = Input(shape=(15, 20))

# kernel = np.random.random((60, ))
# concatenated_vector = concatenate_and_multiply(v, w)
# result = dot([kernel, concatenated_vector], axes=-1)
# print(result.shape)

In [ ]:
# import numpy as np 
# from keras import backend as K 

# A = np.random.random((1000,32,64,3))
# # B = np.moveaxis( A, 3, 1)
# C = np.transpose( A, (0,3,1,2))

# print(A.shape)
# print(C.shape)

# A_t = K.variable(A)
# C_t = K.permute_dimensions(A_t, (0,3,1,2))

# print(K.eval(A_t).shape)
# print(K.eval(C_t).shape)

In [ ]:
# import numpy as np 
# from keras import backend as K 
# from keras.models import Input, Model
# from keras.layers.advanced_activations import Softmax

# A = np.random.random((10,32,64)).astype('float32')

# i = Input(shape=(32,64), dtype='float32')
# o = Softmax(axis=-1)(i)
# model = Model([i], [o])
# y = model.predict([A])
# print(y)
# print(y.shape)
# print(np.sum(y, axis=-2))
# print(np.sum(y, axis=-1))
# print(np.sum(y, axis=-1).shape)
# print(np.sum(y, axis=-2).shape)

In [ ]:
# import numpy as np 
# from keras import backend as K 

# A = np.random.random((32,6))
# # B = np.moveaxis( A, 3, 1)
# # C = np.transpose( A, (0,3,1,2))
# B = np.random.random((32, 6))

# print(A.shape)
# print(B.shape)

# def mul(x):
#     A, B = x
#     print(K.int_shape(A))
#     print(K.int_shape(B))
#     return A*B

# A_t = K.variable(A)
# B_t = K.variable(B)
# # C_t = K.permute_dimensions(A_t, (0,3,1,2))

# C = K.map_fn(mul, (A_t, B_t), dtype='float32')

# y = K.eval(C)